## Definição do *dataset*

O *dataset utilizado será o "TOWARDS LIMB POSITION INVARIANT MYOELECTRIC PATTERN RECOGNITION USING TIME-DEPENDENT SPECTRAL FEATURES" [1]. Maiores informações podem ser vistas no site: https://www.rami-khushaba.com/electromyogram-emg-repository.html

Neste *dataset* existem 5 posições e 8 movimentos. Algumas questões de projetos foram levadas em consideração:
1. A primeira posição (P1) possui 40.000 *samples* e todas as outras possuem 20.000 *samples*. Desta forma, a posição 1 foi **excluída** do experimento de teste;
2. Cada posição possui 5 tentativas. Desta forma, serão utilizadas todas as tentativas do mesmo movimento como uma soma total de *samples* (todas as tentativas serão concatenadas);
3. Como este experimento resultará em 4 posições e 8 movimentos, será considerado primeiramente um experimento com 32 classes (4 * 8) e um segundo experimento com 8 classes (somando/dispresando as posições do braço).

[1] R. N. Khushaba, Maen Takruri, Jaime Valls Miro, and Sarath Kodagoda, "Towards limb position invariant myoelectric pattern recognition using time-dependent spectral features", Neural Networks, vol. 55, pp. 42-58, 2014. https://doi.org/10.1016/j.neunet.2014.03.010

### Separação das classes

In [1]:
from glob import glob

# exclusão dos arquivos com "Pos1" no nome
arquivos = glob("datasets/EMG_data/Pos[2-5]*.txt")
# índices para as 32 classes considerando as posições de 2 à 5
cl32 = {
    "Pos2_HandOpen": list(), "Pos3_HandOpen": list(), "Pos4_HandOpen": list(), "Pos5_HandOpen": list(),
    "Pos2_HandRest": list(), "Pos3_HandRest": list(), "Pos4_HandRest": list(), "Pos5_HandRest": list(),
    "Pos2_ObjectGrip": list(), "Pos3_ObjectGrip": list(), "Pos4_ObjectGrip": list(), "Pos5_ObjectGrip": list(),
    "Pos2_PichGrip": list(), "Pos3_PichGrip": list(), "Pos4_PichGrip": list(), "Pos5_PichGrip": list(),
    "Pos2_WristExten": list(), "Pos3_WristExten": list(), "Pos4_WristExten": list(), "Pos5_WristExten": list(),
    "Pos2_WristFlex": list(), "Pos3_WristFlex": list(), "Pos4_WristFlex": list(), "Pos5_WristFlex": list(),
    "Pos2_WristPron": list(), "Pos3_WristPron": list(), "Pos4_WristPron": list(), "Pos5_WristPron": list(),
    "Pos2_WristSupi": list(), "Pos3_WristSupi": list(), "Pos4_WristSupi": list(), "Pos5_WristSupi": list(),
}
# índices para as 8 classes desconsiderando as posições de 2 à 5
cl08 = {
    "HandOpen": list(),
    "HandRest": list(),
    "ObjectGrip": list(),
    "PichGrip": list(),
    "WristExten": list(),
    "WristFlex": list(),
    "WristPron": list(),
    "WristSupi": list(),
}

## Carregamento do *dataset*

Neste trecho de código, todos as amostras serão dividas entre os conjuntos de calsses específicas

In [2]:
import re

for arquivo in arquivos:
    trial_file = open(arquivo)
    nome = trial_file.name.split('/')[-1].split('.')[0]
    trial = list()
    for linha in trial_file.readlines():
        # foi necessário substituir os números "int" por "float"
        linha = re.sub(r"(?<=\ )(\d)(?=\ )", r"\1.0", linha)
        # casamento da linha com 7 pontos (1 para cada eletrodo)
        sample = [float(s) for s in re.findall(r"\-?\d\.\d+", linha)]
        trial.append(sample)
    cl08["{}".format(nome[5:-3])].append(trial)
    cl32["Pos{}{}".format(nome[3], nome[4:-3])].append(trial)

Agora será realizado a divisão dos dados e suas respectivas classes em sequência, formando dados em `numpy.array` para X1 (dados EMG das 8 classes), y1 (*labels* das 8 classes), X2 (dados EMG das 32 classes), y2 (*labels* das 32 classes)

In [4]:
import numpy as np
from sklearn.preprocessing import LabelEncoder

X1 = np.array(list(cl08.values()))
X2 = np.array(list(cl32.values()))
# transformando labels categóricos em numéricos
le = LabelEncoder()
y1 = np.array(le.fit_transform(list(cl08.keys())))
y2 = np.array(le.fit_transform(list(cl32.keys())))
# shape: [8 classes, 24 trials, 20000 samples, 7 eletrodos]
print(X1.shape)
# shape: [32 classes, 6 trials, 20000 samples, 7 eletrodos]
print(X2.shape)
print(y1)
print(y2)

X1 = X1.swapaxes(2, 3)
X2 = X2.swapaxes(2, 3)
print(X1.shape)
print(X2.shape)

(8, 24, 20000, 7)
(32, 6, 20000, 7)
[0 1 2 3 4 5 6 7]
[ 0  8 16 24  1  9 17 25  2 10 18 26  3 11 19 27  4 12 20 28  5 13 21 29
  6 14 22 30  7 15 23 31]
(8, 24, 7, 20000)
(32, 6, 7, 20000)


## Segmentação dos dados ###

Com amostras divididas em 10, 20, 40, 50, 80 e 90 partes e com tamanhos da sobreposição de ~10%, ~20%, ~30%, ~40%, ~50%, ~70%, ~90%:

### Informações para segmentação nos dados no domínio do tempo:

##### ~10% de sobreposição:
* 10 partes = (salto = 1848, segmento = 2048);
* 20 partes = (salto = 992,  segmento = 1128);
* 40 partes = (salto = 496,   segmento = 562);
* 50 partes = (salto = 398,   segmento = 436);
* 80 partes = (salto = 248,   segmento = 272);
* 90 partes = (salto = 220,   segmento = 244).

##### ~20% de sobreposição:
* 10 partes = (salto = 1920, segmento = 2400);
* 20 partes = (salto = 986,  segmento = 1240);
* 40 partes = (salto = 488,  segmento =  608);
* 50 partes = (salto = 396,  segmento =  492);
* 80 partes = (salto = 248,  segmento =  312); 
* 90 partes = (salto = 276,  segmento =  220).

##### ~30% de sobreposição:
* 10 partes = (salto = 1880, segmento = 2648);
* 20 partes = (salto = 966,  segmento = 1380);
* 40 partes = (salto = 492,  segmento =  702);
* 50 partes = (salto = 392,  segmento =  560);
* 80 partes = (salto = 248,  segmento =  352); 
* 90 partes = (salto = 220,  segmento =  312);

##### ~40% de sobreposição:
* 10 partes = (salto = 1766,segmento = 2944);
* 20 partes = (salto = 924, segmento = 1540);
* 40 partes = (salto = 484, segmento = 808 );
* 50 partes = (salto = ?, segmento = ? );
* 80 partes = (salto = ?, segmento = ? ); 
* 90 partes = (salto = ?, segmento = ? );

##### ~50% de sobreposição:
* 10 partes = (salto = ?, segmento = ? );
* 20 partes = (salto = ?, segmento = ? );
* 40 partes = (salto = ?, segmento = ? );
* 50 partes = (salto = ?, segmento = ? );
* 80 partes = (salto = ?, segmento = ? ); 
* 90 partes = (salto = ?, segmento = ? );

##### ~70% de sobreposição:
* 10 partes = (salto = ?, segmento = ? );
* 20 partes = (salto = ?, segmento = ? );
* 40 partes = (salto = ?, segmento = ? );
* 50 partes = (salto = ?, segmento = ? );
* 80 partes = (salto = ?, segmento = ? ); 
* 90 partes = (salto = ?, segmento = ? );

##### ~90% de sobreposição:
* 10 partes = (salto = ?, segmento = ? );
* 20 partes = (salto = ?, segmento = ? );
* 40 partes = (salto = ?, segmento = ? );
* 50 partes = (salto = ?, segmento = ? );
* 80 partes = (salto = ?, segmento = ? ); 
* 90 partes = (salto = ?, segmento = ? );

In [5]:
print(X1.shape)
# definição do salto e do tamanho do segmento (segmento - salto = sobreposição)
salto = 484
segmento = 808
n_win = int((X1.shape[-1] - segmento) / salto) + 1
ids = np.arange(n_win) * salto
x_8 = np.array([X1[:,:,:,k:(k + segmento)] for k in ids]).transpose(1, 2, 3, 0, 4)
print(x_8.shape)

print(X2.shape)
# definição do salto e do tamanho do segmento (segmento - salto = sobreposição)
n_win = int((X2.shape[-1] - segmento) / salto) + 1
ids = np.arange(n_win) * salto
x_32 = np.array([X2[:,:,:,k:(k + segmento)] for k in ids]).transpose(1, 2, 3, 0, 4)
print(x_32.shape)

(8, 24, 7, 20000)
(8, 24, 7, 40, 808)
(32, 6, 7, 20000)
(32, 6, 7, 40, 808)


### Caracteristicas no domínio do tempo:

* `Variance of EMG (VAR)`: 
    > $\frac{1}{N-1}\sum_{i=1}^{N}x_i^2$
* `Root Mean Square (RMS)`:
    > $\sqrt{\frac{1}{N-1}\sum_{i=1}^{N}|x_i|^2}$
* `Mean Absolute Value (MAV)`:
    > $\frac{1}{N}\sum_{i=1}^{N}|x_i|$
* `Zero Crossing (ZC)`:
    > $\sum_{i=1}^{N}sgn(-x_i x_{i+1})$
  onde sgn(X) = 1 se X > 0 | 0 caso contrário
* `Waveform length (WL)`:
    > $\sum_{i=1}^{N-1}|x_i-x_{i+1}|$
* `Simple Square Integral (SSI)`:
    > $\sum_{i=1}^{N}x_i^2$

In [49]:
# VAR
var_8  = np.sum(x_8 ** 2, axis=-1) / (np.prod(x_8.shape[:-1]) - 1)
var_32 = np.sum(x_32 ** 2, axis=-1) / (np.prod(x_32.shape[:-1]) - 1)
print(var_8.shape)
print(var_32.shape)

# RMS
rms_8  = np.sqrt(np.sum(np.abs(x_8) ** 2, axis=-1) / (np.prod(x_8.shape[:-1]) - 1))
rms_32 = np.sqrt(np.sum(np.abs(x_32) ** 2, axis=-1) / (np.prod(x_32.shape[:-1]) - 1))
print(rms_8.shape)
print(rms_32.shape)

# MAV
mav_8  = np.sum(np.abs(x_8), axis=-1) / (np.prod(x_8.shape[:-1]))
mav_32 = np.sum(np.abs(x_32), axis=-1) / (np.prod(x_32.shape[:-1]))
print(mav_8.shape)
print(mav_32.shape)

# ZC
def zc(x):
    for ind, value in np.ndenumerate(x):
        i, j, k, l, m = ind
        if (m+1 < x.shape[-1]): x[ind] = (1 if (((value*(-1))*x[i, j, k, l, m+1]) > 0) else 0)
        else:
            x[ind] = 0
    return x

#zc_8 = zc(x_8)                    #Descomentar para executar ZC na primeira vez
#zc_8 = np.sum(zc_8, axis=-1)      #Descomentar para executar ZC (demorando um pouco..)
print(zc_8.shape)
#zc_32 = zc(x_32)                  #Descomentar para executar ZC
#zc_32 = np.sum(zc_32, axis=-1)    #Descomentar para executar ZC
print(zc_32.shape)

#WL
wl_8 = np.sum(np.abs(np.diff(x_8, axis=-1)), axis=-1)
print(wl_8.shape)
wl_32 = np.sum(np.abs(np.diff(x_32, axis=-1)), axis=-1)
print(wl_32.shape)

#SSI
ssi_8 = np.sum(x_8 ** 2, axis=-1)
ssi_32 = np.sum(x_32 ** 2, axis=-1)
print(ssi_8.shape)
print(ssi_32.shape)

(8, 24, 7, 40)
(32, 6, 7, 40)
(8, 24, 7, 40)
(32, 6, 7, 40)
(8, 24, 7, 40)
(32, 6, 7, 40)
(8, 24, 7, 40)
(32, 6, 7, 40)
(8, 24, 7, 40)
(32, 6, 7, 40)
(8, 24, 7, 40)
(32, 6, 7, 40)


##### Transformação para domínio da frequencia

##### ~10% de sobreposição:
* 10 partes = (nperseg = 2560, noverlap = 256);
* 20 partes = (nperseg = 1216, noverlap = 128);
* 40 partes = (nperseg = 586,   noverlap = 60);
* 50 partes = (nperseg = 464,   noverlap = 48);
* 80 partes = (nperseg = 282,   noverlap = 28);
* 90 partes = (nperseg = 250,   noverlap = 24).

##### ~20% de sobreposição:
* 10 partes = (nperseg = 2816, noverlap = 564);
* 20 partes = (nperseg = 1344, noverlap = 268);
* 40 partes = (nperseg = 656,  noverlap = 132);
* 50 partes = (nperseg = 520,  noverlap = 104);
* 80 partes = (nperseg = 318,   noverlap = 64);
* 90 partes = (nperseg = 282,   noverlap = 56).

##### ~30% de sobreposição:
* 10 partes = (nperseg = 3200, noverlap = 960);
* 20 partes = (nperseg = 1536, noverlap = 460);
* 40 partes = (nperseg = 736,  noverlap = 220);
* 50 partes = (nperseg = 592,  noverlap = 178);
* 80 partes = (nperseg = 362,  noverlap = 108);
* 90 partes = (nperseg = 324,   noverlap = 98).

##### ~40% de sobreposição:
* 10 partes = (nperseg = , noverlap = );
* 20 partes = (nperseg = , noverlap = );
* 40 partes = (nperseg = ,   noverlap = );
* 50 partes = (nperseg = ,   noverlap = );
* 80 partes = (nperseg = ,   noverlap = );
* 90 partes = (nperseg = ,   noverlap = ).

##### ~50% de sobreposição:
* 10 partes = (nperseg = , noverlap = );
* 20 partes = (nperseg = , noverlap = );
* 40 partes = (nperseg = ,   noverlap = );
* 50 partes = (nperseg = ,   noverlap = );
* 80 partes = (nperseg = ,   noverlap = );
* 90 partes = (nperseg = ,   noverlap = )..

##### ~70% de sobreposição:
* 10 partes = (nperseg = , noverlap = );
* 20 partes = (nperseg = , noverlap = );
* 40 partes = (nperseg = ,   noverlap = );
* 50 partes = (nperseg = ,   noverlap = );
* 80 partes = (nperseg = ,   noverlap = );
* 90 partes = (nperseg = ,   noverlap = )..

##### ~90% de sobreposição:
* 10 partes = (nperseg = , noverlap = );
* 20 partes = (nperseg = , noverlap = );
* 40 partes = (nperseg = ,   noverlap = );
* 50 partes = (nperseg = ,   noverlap = );
* 80 partes = (nperseg = ,   noverlap = );
* 90 partes = (nperseg = ,   noverlap = ).



In [50]:
from scipy.signal import stft

print(X1.shape)
_, _, w = stft(X1, fs=4000, nperseg=736, noverlap=220)
w = np.swapaxes(w, 3, 4)
print(w.shape)

print(X2.shape)
_, _, z = stft(X2, fs=4000, nperseg=1216, noverlap=128)
z = np.swapaxes(z, 3, 4)
print(z.shape)

(8, 24, 7, 20000)
(8, 24, 7, 40, 369)
(32, 6, 7, 20000)
(32, 6, 7, 20, 609)


### Características no domínio da frequencia

* `Frequency Median (FMD)`:
    > $\frac{1}{2}\sum_{i=1}^{M}PSD$
* `Modified Median Frequency (MMDF)`:
    > $\frac{1}{2}\sum_{i=1}^{M}A_j$
* `Mean Power (MP)`:
    > $\frac{1}{M}\sum_{i=1}^{M}P_i$
* `Peak Frequency (PKF)`:
    > $max(P_i), i=1,...,M$

In [51]:
# definição da função PSD para o sinal no domínio da frequência
def PSD(x):
    return np.sqrt(np.abs(x))

# FMD
fmd_8 = np.sum(PSD(w), axis=-1) / 2
fmd_32 = np.sum(PSD(z), axis=-1) / 2
print(fmd_8.shape)
print(fmd_32.shape)

# MMDF
mmdf_8 = np.sum(np.abs(w), axis=-1) / 2
mmdf_32 = np.sum(np.abs(z), axis=-1) / 2
print(mmdf_8.shape)
print(mmdf_32.shape)

#MP
mp_8 = np.real(np.sum(w, axis=-1) / (np.prod(w.shape[:-1]))) 
mp_32 = np.real(np.sum(z, axis=-1) / (np.prod(z.shape[:-1]))) 
print(mp_8.shape)
print(mp_32.shape)

#PKF
pkf_8 = np.real(np.max(w, axis=-1))
pkf_32 = np.real(np.max(z, axis=-1))
print(pkf_8.shape)
print(pkf_32.shape)


(8, 24, 7, 40)
(32, 6, 7, 20)
(8, 24, 7, 40)
(32, 6, 7, 20)
(8, 24, 7, 40)
(32, 6, 7, 20)
(8, 24, 7, 40)
(32, 6, 7, 20)


### Vetor de características

In [56]:
features = list()
#Implementar 'COMBINATIONS' com as caracteristicas
for feature in (fmd_8, mmdf_8, mp_8, pkf_8):
    feature = feature.transpose(0, 1, 3, 2)
    feature = feature.reshape(var_8.shape[0] * var_8.shape[1] * var_8.shape[-1], 7)
    print('Feature: {}'.format(feature), feature.shape)
    features.append(feature)

X = np.concatenate(features, axis=-1)
X.shape

#features = list()
#for feature in (fmd_32, mp_32):
#    feature = feature.transpose(0, 1, 3, 2)
#    feature = feature.reshape(var_32.shape[0] * var_32.shape[1] * var_32.shape[-1], 7)
#    print('Feature: {}'.format(feature), feature.shape)
#    features.append(feature)
#
#X = np.concatenate(features, axis=-1)
#X.shape

Feature: [[2.71594727 3.47711758 9.77935422 ... 9.36951927 3.49331284 3.57303598]
 [2.90849263 4.04003035 9.84606439 ... 7.92276728 3.93537543 2.98444844]
 [2.86594703 3.95873071 8.35683735 ... 7.71016319 3.8868443  2.85104305]
 ...
 [2.78067935 3.99210657 8.72368614 ... 6.47208419 3.61067164 2.99245604]
 [2.82207889 3.80862371 9.04968927 ... 5.85300005 3.38188101 2.85588315]
 [1.9857196  3.51256975 9.22378052 ... 4.18843877 3.38770887 2.40031102]] (7680, 7)
Feature: [[0.0842384  0.13714626 1.05278827 ... 1.0010851  0.19369028 0.11529584]
 [0.10392894 0.25996687 1.57037235 ... 1.1135238  0.26204607 0.11637709]
 [0.10120799 0.24671305 1.09727838 ... 1.0959483  0.23909547 0.11193189]
 ...
 [0.09777364 0.26440842 1.30292832 ... 0.70801129 0.18597263 0.12042253]
 [0.09716157 0.22812454 1.39545271 ... 0.55259207 0.16274498 0.1072439 ]
 [0.05032287 0.13590309 0.61916389 ... 0.30300867 0.08653622 0.05712587]] (7680, 7)
Feature: [[ 2.04211515e-09  4.36427070e-08 -5.30788233e-08 ... -5.08602930

(7680, 28)

Vetor de labels

In [57]:
y = np.array([[str(i)] * int(X.shape[0] / 8) for i in range(8)]) #switch 8 - 32
y = y.reshape(y.shape[0] * y.shape[1])
y.shape

(7680,)

## Classificador SVM

In [58]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

# treino e teste em 70 e 30% respectivamente
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True)

# testar outros classificadores e definir parametrização
clf = SVC(gamma='auto')
clf.fit(X_train, y_train)

SVC(gamma='auto')

Rotina Simples

In [59]:
res = clf.predict(X_test)
tot_hit = sum([1 for i in range(len(res)) if res[i] == y_test[i]])
print('Acurácia: {:.2f}%'.format(tot_hit / X_test.shape[0] * 100))

Acurácia: 94.10%


### Resultados parciais com as 8 classe, 4 características e ~10% de sobreposição:

###### 10 partes: 93.23%
###### 20 partes: 91.75%
###### 40 partes: 91.88%
###### 50 partes: 90.31%
###### 80 partes: 87.33%
###### 90 partes: 85.47%

### Resultados parciais com as 32 classe, 4 características e ~10% de sobreposição:

###### 10 partes: 82.47%
###### 20 partes: 84.90%
###### 40 partes: 82.55%
###### 50 partes: 78.82%
###### 80 partes: 73.83%
###### 90 partes: 70.93%